<a href="https://colab.research.google.com/github/PierreCasco/twitter_and_football/blob/master/Twitter_and_Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tweepy
import jsonpickle

In [0]:
CONSUMER_KEY = 'aGWJPayGiJNjtp6VspbiRMPMO'
CONSUMER_SECRET = '69F1C2TVR9A8lcqfcmcYiTXOrXuUgyKdySQTgS6i48PF9P7eM7'
OAUTH_TOKEN = '323252972-LtPGnQLLjTyvHPaB1Kz5btKTpG0RUxu6yxvYGpsp'
OAUTH_SECRET = 'v5DaZVTrPUSVTF4X91Xe6fYMQ74tZpUxIqpYoP2mxQwXY'

In [0]:
def oauth_login():
  # get the authorization from Twitter and save in the Tweepy package
  auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
  auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
  tweepy_api = tweepy.API(auth)

    # if a null api is returned, give error message
  if (not tweepy_api):
      print ("Problem Connecting to API with OAuth")

  # return the Twitter api object that allows access for the Tweepy api functions
  return tweepy_api

# login to Twitter with extended rate limiting
#  must be used with the Tweepy Cursor to wrap the search and enact the waits
def appauth_login():
  # get the authorization from Twitter and save in the Tweepy package
  auth = tweepy.AppAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
  # apparently no need to set the other access tokens
  tweepy_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

  # if a null api is returned, give error message
  if (not tweepy_api):
      print ("Problem Connecting to API with AppAuth")

  # return the Twitter api object that allows access for the Tweepy api functions
  return tweepy_api

In [5]:
if __name__ == '__main__':
  tweepy_api = oauth_login()
  print ("Twitter OAuthorization: ", tweepy_api)
  tweepy_api = appauth_login()
  print ("Twitter AppAuthorization: ", tweepy_api)

Twitter OAuthorization:  <tweepy.api.API object at 0x7f4331f30908>
Twitter AppAuthorization:  <tweepy.api.API object at 0x7f4348d4c0f0>


In [0]:
def get_save_tweets(filepath, api, query, max_tweets=1000000, lang='pt'):

    tweetCount = 0

    #Open file and save tweets
    with open(filepath, 'w') as f:

        # Send the query
        for tweet in tweepy.Cursor(api.search,q=query,lang=lang).items(max_tweets):         

            #Convert to JSON format
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

        #Display how many tweets we have collected
        print("Downloaded {0} tweets".format(tweetCount)) 

In [0]:
query = '#NFLKickoff OR #ThursdayNightFootball OR #NFL100 OR #GoPackGo OR #Bears100 OR #BearsVsPackers'

# Get those tweets
get_save_tweets('tweets.json', tweepy_api, query)

In [0]:
def tweets_to_df(path):
    
    tweets = list(open('tweets.json', 'rt'))
    
    text = []
    weekday = []
    month = []
    day = []
    hour = []
    hashtag = []
    url = []
    favorite = []
    reply = []
    retweet = []
    follower = []
    following = []
    user = []
    screen_name = []

    for t in tweets:
        t = jsonpickle.decode(t)
        
        # Text
        text.append(t['text'])
        
        # Decompose date
        date = t['created_at']
        weekday.append(date.split(' ')[0])
        month.append(date.split(' ')[1])
        day.append(date.split(' ')[2])
        
        time = date.split(' ')[3].split(':')
        hour.append(time[0]) 
        
        # Has hashtag
        if len(t['entities']['hashtags']) == 0:
            hashtag.append(0)
        else:
            hashtag.append(1)
            
        # Has url
        if len(t['entities']['urls']) == 0:
            url.append(0)
        else:
            url.append(1)
            
        # Number of favs
        favorite.append(t['favorite_count'])
        
        # Is reply?
        if t['in_reply_to_status_id'] == None:
            reply.append(0)
        else:
            reply.append(1)       
        
        # Retweets count
        retweet.append(t['retweet_count'])
        
        # Followers number
        follower.append(t['user']['followers_count'])
        
        # Following number
        following.append(t['user']['friends_count'])
        
        # Add user
        user.append(t['user']['name'])

        # Add screen name
        screen_name.append(t['user']['screen_name'])
        
    d = {'text': text,
         'weekday': weekday,
         'month' : month,
         'day': day,
         'hour' : hour,
         'has_hashtag': hashtag,
         'has_url': url,
         'fav_count': favorite,
         'is_reply': reply,
         'retweet_count': retweet,
         'followers': follower,
         'following' : following,
         'user': user,
         'screen_name' : screen_name
        }
    
    return pd.DataFrame(data = d)
        
tweets_df = tweets_to_df('tweets.json')

In [104]:
tweets_df

,text,weekday,month,day,hour,has_hashtag,has_url,fav_count,is_reply,retweet_count,followers,following,user,screen_name
0,"RT @NFLBrasil: ESSE MOMENTO É SEU, FÃ DE FUTEB...",Fri,Sep,06,00,1,0,0,0,144,87,194,Chari,chaarilima_
1,Aí que lindo todos de roupinhas dos Anos 20 😍😍...,Fri,Sep,06,00,1,0,0,0,0,725,2113,Priscylla Gonçalves,Pri_GRibeiro
2,🏈 OS PRIMEIROS TRÊS! 🏆 \n\nO Green Bay abre o ...,Fri,Sep,06,00,0,1,0,1,0,1180,271,FD Sports,FDSportsOfc
3,RT @piadas_nfl: QUE DELÍCIA! 😍😍😍\n\n #NFL100 !...,Fri,Sep,06,00,1,0,0,0,9,206,230,lari,Larissa_lfc
4,RT @NFLBrasil: A DEFESA DOS BEARS NÃO VEIO PAR...,Fri,Sep,06,00,1,0,0,0,23,24,122,Gustavo Diniz,DinizGustavo10
5,É um baita presente de aniversário ter a volta...,Fri,Sep,06,00,1,0,0,0,0,476,389,Bruno Lima,revirabruno
6,Os ataques estão horríveis. Parece o último Su...,Fri,Sep,06,00,1,0,0,0,0,816,405,Isaías Augusto #GoPats #ForaLeco,isaiasaugusto
7,RT @lfernando86: Aproveitando o intervalo pra ...,Fri,Sep,06,00,0,0,0,0,3,1060,140,45jardas,45jardas
8,RT @NFLBrasil: Um pedido de casamento ao vivo ...,Fri,Sep,06,00,1,0,0,0,18,167,324,Vinicius Costa,viny__BC
9,"RT @NFLBrasil: ESSE MOMENTO É SEU, FÃ DE FUTEB...",Fri,Sep,06,00,1,0,0,0,144,7127,387,Guilherme Salles,sallesgui93
